# Homework 4 (Part II)

Aprendizagem 2023/2024 - LEIC @ IST 

Group #24
- Daniel Nunes (Nº 103095)
- Gonçalo Alves (Nº 103540)

## Data importing and normalization

In [5]:
import numpy as np
from scipy.io import arff
from sklearn.preprocessing import MinMaxScaler

# Load your ARFF file
data, meta = arff.loadarff('column_diagnosis.arff')

# Convert the ARFF data to a NumPy array
data = np.array(data.tolist())

# Extract the numeric features (the first 6 columns) for normalization
numeric_data = data[:, :-1].astype(float)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data using the scaler
normalized_data = scaler.fit_transform(numeric_data)

display(normalized_data)

array([[0.35568788, 0.51989984, 0.22917997, 0.2508573 , 0.30746116,
        0.02514839],
       [0.12450104, 0.2967831 , 0.09857833, 0.14462935, 0.47664891,
        0.03636497],
       [0.41166648, 0.51393229, 0.32299466, 0.30766054, 0.38609692,
        0.0175229 ],
       ...,
       [0.34043781, 0.52244298, 0.28789745, 0.23490726, 0.59779618,
        0.01943732],
       [0.18425678, 0.27235174, 0.24684569, 0.21462279, 0.52117504,
        0.02624045],
       [0.07420202, 0.20770855, 0.20261992, 0.14251659, 0.57924032,
        0.02527676]])

## Exercise 1

Using sklearn, apply k-means clustering fully unsupervisedly on the normalized data with 
𝑘 ∈{2,3,4,5} (random=0 and remaining parameters as default). Assess the silhouette and purity of 
the produced solutions

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Define the range of k values to try
k_values = [2, 3, 4, 5]

# Perform k-means clustering for different values of k
for k in k_values:
    kmeans = KMeans(n_clusters=8, random_state=0, n_init='auto') # the default value of n_clusters is 8 
    cluster_labels = kmeans.fit_predict(normalized_data)
    
    # Calculate silhouette score
    silhouette_avg = silhouette_score(normalized_data, cluster_labels)
    print(f"Silhouette score for k={k}: {silhouette_avg}")
    
    # Assess purity
    # You need to know the ground truth labels to calculate purity
    # If you don't have ground truth labels, purity cannot be calculated
    ground_truth_labels = data[:, -1].astype(str)
    le = LabelEncoder()
    ground_truth_labels_encoded = le.fit_transform(ground_truth_labels)
    
    # Assuming cluster_labels and ground_truth_labels_encoded have the same length
    accuracy = accuracy_score(ground_truth_labels_encoded, cluster_labels)
    print(f"Accuracy (Purity) for k={k}: {accuracy}")

Silhouette score for k=2: 0.20730150522479207
Accuracy (Purity) for k=2: 0.02258064516129032
Silhouette score for k=3: 0.20730150522479207
Accuracy (Purity) for k=3: 0.02258064516129032
Silhouette score for k=4: 0.20730150522479207
Accuracy (Purity) for k=4: 0.02258064516129032
Silhouette score for k=5: 0.20730150522479207
Accuracy (Purity) for k=5: 0.02258064516129032


## Exercise 2

Consider the application of PCA after the data normalization:

i. Identify the variability explained by the top two principal components.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principal_components = pca.fit_transform(normalized_data)

explained_var_ratio = pca.explained_variance_ratio_
print(f"The variability of the top two principal components \n" +
      "is equal to", explained_var_ratio)

# TODO: Verify this exercise

The variability of the top two principal components 
is equal to [0.56181445 0.20955953]


ii. For each one of these two components, sort the input variables by relevance by inspecting the absolute weights of the linear projection.

**Our answer**

## Exercise 3

Visualize side-by-side the data using: i) the ground diagnoses, and ii) the previously learned k=3 clustering solution. To this end, projected the normalized data onto a 2-dimensional data space using PCA and then color observations using the reference and cluster annotations. 

## Exercise 4

Considering the results from questions (1) and (3), identify two ways on how clustering can be used to characterize the population of ill and healthy individuals. 

**Our answer**

We can conclude from this data that, as the number of maximum iterations for training a model increases, its RMSE values decrease, meaning that the model gets more accurate as the amount of training increases.

However, too many iterations may start to cause overfitting on our model, meaning that, while it follows its predictions closer to the training data, it can be more error-prone when evaluating completely new sets of data. Hence, the early-stopping strategy tries to strike a balance between the model's accuracy to the training data and the possibility of overfitting, favouring performance in most cases. This might be the reason why its RMSE value is slightly larger than the one calculated from the model that stops training after 200 iterations.